# Blind spot map
Data Preparation

In [21]:
import pandas as pd
import numpy as np

#df = pd.read_excel("https://raw.githubusercontent.com/laurens-kolenbrander/DEiA-2/blob/73ac2e4abc9fda81321891ed8de4da258670c27c/2019_cbs_data.xls?token=GHSAT0AAAAAABUAYSV2GPBNRDKI4XE6TY2UYTPOCYA")
df = pd.read_excel("2019_cbs_data.xls")

regio_type = ['Land','Gemeente', 'Wijk']
df = df[~df.recs.isin(regio_type)]

df.head()

from_scrath = False

,gwb_code_10,gwb_code_8,regio,gm_naam,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,...,g_afs_kv,g_afs_sc,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad
3,BU00030000,30000,Appingedam-Centrum,Appingedam,Buurt,BU00030000,1,2370,1135,1230,...,"1,2","1,1","7,1",90,84,5,9901,1,3,1210
4,BU00030001,30001,Appingedam-West,Appingedam,Buurt,BU00030001,1,3035,1505,1530,...,"1,0","1,0","4,0",163,158,5,9903,6,4,903
5,BU00030002,30002,Appingedam-Oost,Appingedam,Buurt,BU00030002,1,5725,2760,2960,...,"0,7","0,7","7,7",295,284,11,9902,1,3,1140
6,BU00030007,30007,Verspreide huizen Damsterdiep en Eemskanaal,Appingedam,Buurt,BU00030007,1,325,175,150,...,"1,3","1,3","3,5",559,540,18,9903,4,5,359
7,BU00030008,30008,Verspreide huizen ten zuiden van Eemskanaal,Appingedam,Buurt,BU00030008,1,100,55,45,...,"2,7","2,7","2,7",582,554,28,9902,1,5,79


In [2]:
# Import of neighbourhood distances
distances = pd.read_csv("neighbourhood_distances.csv")


In [50]:
def minimal_distance(distances = distances):
    short = distances[['neighbourhood', 'distance', 'category']]
    
    sport = short[short['category']=='sport']
    park = short[short['category']=='park']
    playground = short[short['category']=='playground']
    neighbourhoods = short['neighbourhood']
    filt = neighbourhoods.drop_duplicates()

    for i in filt:
        single = sport[sport['neighbourhood']==i]
        single = single.sort_values(by=['distance'])
        #print(single.iloc[0]['distance'])
        sportmin = single.iloc[0]['distance']

        psingle = park[park['neighbourhood']==i]
        psingle = psingle.sort_values(by=['distance'])
        parkmin = psingle.iloc[0]['distance']

        plsingle = playground[playground['neighbourhood']==i]
        plsingle = plsingle.sort_values(by=['distance'])
        playmin = plsingle.iloc[0]['distance']

        df2 = pd.DataFrame([[i, sportmin, playmin, parkmin]], columns=['neighbourhood', 'sport_distance', 'play_distance','park_distance'])

        minimals = pd.concat([minimals, df2])
        
    return minimals

if from_scrath:
    minimals = minimal_distance(distances)
    # Save to csv
    minimals.to_csv('minimaldistances_concat.csv')
else:
    # Importing minimal distance
    minimals = pd.read_csv('minimaldistances_concat.csv')
    minimals = minimals.drop(columns='Unnamed: 0')

In [24]:
df_filtered = df[['regio', # neighbourhood
                  'gm_naam', # municipality, only for simple reading afterwards
                  'a_inw', #inhabitants
                  'a_vrouw', #women amount
                  'a_00_14', #0-15
                  'a_15_24', #15-24
                  'a_25_44', #25-44
                  'a_45_64', #45-64
                  'a_geb', #births
                  'a_hh_m_k', # households with children
                  'a_ongeh', #unmarried
                  'p_huurw',  # perc. rental 
                  'p_arb_pp', # perc. nett working participaton
                  'a_opl_lg',  # amount low education
                  'a_opl_md',  #amount mid edu
                  'a_opl_hg', # amount high edu
                  'g_hh_sti', # average standardized income
                  'g_pau_hh', #cars per hh
                  'pst_mvp' # most occuring postal
                 ]]


df_renamed = df_filtered.rename(columns={'regio': 'neighbourhood', 
                                         'gm_naam': 'municipality',
                                         'a_inw': 'citizens', 
                                         'a_vrouw': 'females', 
                                         'a_ongeh': 'not_married', 
                                         'p_huurw': 'rental_perc', 
                                         'p_arb_pp': 'percentage_working', 
                                         'a_opl_lg': 'low_edu',  
                                         'a_opl_md': 'mid_edu',  
                                         'a_opl_hg': 'high_edu', 
                                         'a_geb': 'births',
                                         'g_hh_sti': 'avg_income',
                                         'a_hh_m_k': 'households_with_children', 
                                         'g_pau_hh': 'cars_per_household', 
                                         'pst_mvp': 'postal_code_center'
                                        })

df_renamed.head()

,neighbourhood,municipality,citizens,females,a_00_14,a_15_24,a_25_44,a_45_64,births,households_with_children,not_married,rental_perc,percentage_working,low_edu,mid_edu,high_edu,avg_income,cars_per_household,postal_code_center
3,Appingedam-Centrum,Appingedam,2370,1230,230,225,500,715,10,250,985,59,61,480,650,720,.,"0,9",9901
4,Appingedam-West,Appingedam,3035,1530,480,350,570,990,25,480,1285,31,66,680,1150,520,.,"1,2",9903
5,Appingedam-Oost,Appingedam,5725,2960,900,580,1205,1560,40,865,2415,60,57,1460,2200,440,"24,2","0,9",9902
6,Verspreide huizen Damsterdiep en Eemskanaal,Appingedam,325,150,65,45,65,115,0,60,170,5,78,40,140,70,.,"1,8",9903
7,Verspreide huizen ten zuiden van Eemskanaal,Appingedam,100,45,10,15,10,45,0,15,40,12,.,40,30,10,.,.,9902


In [6]:
def percentage_missing(df = df_renamed, missing_value = '.', percentage = True):
    ''' Counts missing values as specified in missing_value.
    Optionally percentage can be set to False for the count of values
    '''
    total = df.shape[0]
    for column in df.columns:
        count = df[df[column] == missing_value].shape[0]
        if count != 0:
            if percentage:
                print(f'{column}: {round(count/total*100)}%')
            else:
                print(f'{column}: {count}')

percentage_missing(df_renamed, '.', percentage = True)

rental_perc: 11%
percentage_working: 26%
low_edu: 14%
mid_edu: 14%
high_edu: 14%
avg_income: 97%
cars_per_household: 21%
postal_code_center: 1%


In [61]:
for column in df_renamed.columns:
    if (not column  in ('neighbourhood','municipality')):
        df_renamed[column] = df_renamed[column].replace('.', 0).apply(str).str.replace(',', '.').astype(float)

In [63]:
from sklearn.preprocessing import MinMaxScaler

def demand(df_renamed = df_renamed, a_00_14_weight = 1, households_with_children_weight = 1, cars_weight = 1, births_weight = 1, rental_weight = 1,
            a_45_64_weight = 1, females_weight = 1, single_weight = 1, low_edu_weight = 1, high_edu_weight = 1, working_weight = 1, citizens_weight = 1,
            a_00_14_quantile = 0.5, households_with_children_quantile = 0.5, cars_quantile = 0.5, births_quantile = 0.5, rental_quantile = 0.5,
            a_45_64_quantile = 0.5, females_quantile = 0.5, singles_quantile = 0.5, low_edu_quantile = 0.5, high_edu_quantile = 0.5, working_quantile = 0.5, citizens_quantile = 0.5):
    ''' When a variable is not to be considered the weight should be set to zero
    For higher weight, >1  and lower 0 < weight < 1
    
    Quantiles set the demand for each variable. When higher, only higher values will be taken into account for demand calculation (and vice versa).
    '''
    data = df_renamed.copy()
    # Children under 15
    data['demand_a_00_14']= data['a_00_14']\
        .apply(lambda x: a_00_14_weight*1 if (x > data['a_00_14'].quantile(a_00_14_quantile)) else 0 )
    # + 45 < Age < 64
    data['demand_a_45_64']= data['a_45_64']\
        .apply(lambda x: a_45_64_weight*1 if (x > data['a_45_64'].quantile(a_45_64_quantile)) else 0 )
    # Households with children
    data['demand_households_with_children']= data['households_with_children']\
        .apply(lambda x: households_with_children_weight*1 if (x > data['households_with_children'].quantile(households_with_children_quantile)) else 0 )
    # cars per household
    data['demand_cars']= data['cars_per_household']\
        .apply(lambda x: cars_weight*1 if (x > data['cars_per_household'].quantile(cars_quantile)) else 0 )
    # births
    data['demand_births']= data['births']\
        .apply(lambda x: births_weight*1 if (x > data['births'].quantile(births_quantile)) else 0 )
    # rental homes
    data['demand_rental']= data['rental_perc']\
        .apply(lambda x: rental_weight*1 if (x > data['rental_perc'].quantile(rental_quantile)) else 0 )
    # + %male
    data['demand_females_perc']= data['females']/data['citizens']
    data['demand_females_perc']= data['demand_females_perc']\
        .apply(lambda x: females_weight*1 if (x > data['demand_females_perc'].quantile(females_quantile)) else 0 )
    # + %singles
    data['demand_singles_perc']= data['not_married']/data['citizens']
    data['demand_singles_perc']= data['demand_singles_perc']\
        .apply(lambda x: single_weight*1 if (x > data['demand_singles_perc'].quantile(singles_quantile)) else 0 )
    # + education level
    data['demand_low_edu']= data['low_edu']\
        .apply(lambda x: low_edu_weight*1 if (x > data['low_edu'].quantile(low_edu_quantile)) else 0 )
    data['demand_high_edu']= data['high_edu']\
        .apply(lambda x: high_edu_weight*1 if (x > data['high_edu'].quantile(high_edu_quantile)) else 0 )
    # + %unemployed
    data['demand_percentage_working']= data['percentage_working']\
        .apply(lambda x: working_weight*1 if (x > data['percentage_working'].quantile(working_quantile)) else 0 )
    # + town size
    data['demand_citizens']= data['citizens']\
        .apply(lambda x: citizens_weight*1 if (x > data['citizens'].quantile(citizens_quantile)) else 0 )
    return data

if from_scrath:
    df_demand = demand(df_renamed)

# Playground Demand
Playground demand = (% of children <15) 
'+ (households with children (can be a percentage)) 
'+ (amount of births) 
'+ (income of households) 
'- (cars per household) 
'+ (% rental homes) 


In [65]:
def demand_play(data):
    ''' When a variable is not to be considered the weight should be set to zero
    For higher weight, >1  and lower <1
    '''
    data['play_demand'] = data['demand_a_00_14']\
                          + data['demand_households_with_children']\
                          + data['demand_cars']\
                          + data['demand_births']\
                          + data['demand_rental']
    data['play_demand'] = MinMaxScaler().fit_transform(np.array(data['play_demand']).reshape(-1,1))
 
    return data

if from_scrath:
    df_demand = demand_play(df_demand)

## Sports
Sport_demand = - age - presence of children + (%male) + (%singles — rather than married) + household income + education level + (%unemployed) + town size


In [66]:
def demand_sport(data):
    ''' When a variable is not to be considered the weight should be set to zero
    For higher weight, >1  and lower <1
    '''
    data['sport_demand'] = data['demand_a_00_14']\
                          - data['demand_a_45_64']\
                          - data['demand_females_perc']\
                          + data['demand_singles_perc']\
                          - data['demand_households_with_children']\
                          - data['demand_low_edu']\
                          + data['demand_high_edu']\
                          - data['demand_percentage_working']\
                          + data['demand_citizens']
    data['sport_demand'] = MinMaxScaler().fit_transform(np.array(data['sport_demand']).reshape(-1,1))
     
    return data

if from_scrath:
    df_demand = demand_sport(df_demand)

## Parks and Nature
Family with children (+), low-income (-), middle- to high income (+), low-education (-), high-education (+), 0-16 (++), 16-55 (+), >55 (-), female (+), single (+), distance > 1000m (+) 


In [67]:
def demand_park(data):
    # calculated sum of demand
    data['park_demand'] = data['demand_households_with_children']\
                          - data['demand_low_edu']\
                          + data['demand_high_edu']\
                          + data['demand_a_00_14']\
                          - data['demand_a_45_64']\
                          + data['demand_females_perc']\
                          + data['demand_singles_perc']
    data['park_demand'] = MinMaxScaler().fit_transform(np.array(data['park_demand']).reshape(-1,1))
     
    return data

if from_scrath:
    df_demand = demand_park(df_demand)

In [71]:
def adjust_demand_with_distance(df_demand = df_demand, minimals = minimals, play_distance_min = 0.4, park_distance_min = 1, sport_distance_min = 1):
    df = df_demand.merge(minimals, left_on='neighbourhood', right_on='neighbourhood')

    for index, row in df.iterrows():
        if (row['play_distance'] <= play_distance_min):
            df.loc[index, 'play_demand'] = 0.0

        if (row['park_distance'] <= park_distance_min):
            df.loc[index, 'park_demand'] = 0.0

        if (row['sport_distance'] <= sport_distance_min):
            df.loc[index, 'sport_demand'] = 0.0
            
    return df

if from_scrath:
    df_demand = adjust_demand_with_distance(df_demand, minimals)

In [98]:
if from_scrath:
    # Importing coordinates
    post_coordinate_map = pd.read_csv('postcode_coordinate.csv')

    # This drops 74 rows
    cbs_data_prepped = df_demand[df_demand['postal_code_center'] != '.' ]
    cbs_data_prepped['postal_code_center'] = cbs_data_prepped['postal_code_center'].astype(int)

    with_coordinates = pd.merge(cbs_data_prepped, post_coordinate_map,  how='left', left_on=['postal_code_center'], right_on = ['postcode'])
    
    # Save to csv
    with_coordinates.to_csv('with_coordinates.csv')
    
else:
    # Importing minimal distance
    with_coordinates = pd.read_csv('with_coordinates.csv')
    with_coordinates = with_coordinates.drop(columns='Unnamed: 0')


,neighbourhood,municipality,citizens,females,a_00_14,a_15_24,a_25_44,a_45_64,births,households_with_children,...,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
0,Appingedam-Centrum,Appingedam,2370.0,1230.0,230.0,225.0,500.0,715.0,10.0,250.0,...,4615.0,9901.0,Appingedam,NaN,Appingedam,Groningen,596.0,53.329242,6.850742,Adres
1,Appingedam-West,Appingedam,3035.0,1530.0,480.0,350.0,570.0,990.0,25.0,480.0,...,4617.0,9903.0,Appingedam,NaN,Appingedam,Groningen,596.0,53.308678,6.831403,Adres
2,Appingedam-Oost,Appingedam,5725.0,2960.0,900.0,580.0,1205.0,1560.0,40.0,865.0,...,4616.0,9902.0,Appingedam,NaN,Appingedam,Groningen,596.0,53.307070,6.859370,Adres
3,Verspreide huizen Damsterdiep en Eemskanaal,Appingedam,325.0,150.0,65.0,45.0,65.0,115.0,0.0,60.0,...,4617.0,9903.0,Appingedam,NaN,Appingedam,Groningen,596.0,53.308678,6.831403,Adres
4,Verspreide huizen ten zuiden van Eemskanaal,Appingedam,100.0,45.0,10.0,15.0,10.0,45.0,0.0,15.0,...,4616.0,9902.0,Appingedam,NaN,Appingedam,Groningen,596.0,53.307070,6.859370,Adres
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13487,Streefkerk-Dorp,Molenlanden,1745.0,870.0,290.0,260.0,330.0,490.0,20.0,275.0,...,977.0,2959.0,Streefkerk,NaN,Liesveld,Zuid-Holland,184.0,51.899891,4.744880,Adres
13488,Waal-Dorp,Molenlanden,110.0,55.0,20.0,20.0,25.0,40.0,5.0,20.0,...,983.0,2968.0,Waal,NaN,Liesveld,Zuid-Holland,184.0,51.938185,4.900937,Adres
13489,Wijngaarden-Buitengebied,Molenlanden,10.0,5.0,5.0,5.0,0.0,5.0,0.0,0.0,...,1245.0,3366.0,Wijngaarden,WYNGAARDEN,Graafstroom,Zuid-Holland,184.0,51.844976,4.763978,Adres
13490,Lintbebouwing-Oost,Molenlanden,150.0,80.0,40.0,20.0,20.0,40.0,5.0,25.0,...,1245.0,3366.0,Wijngaarden,WYNGAARDEN,Graafstroom,Zuid-Holland,184.0,51.844976,4.763978,Adres


In [79]:
#Playground output
play_demand_threshold = 0.7
play_output_list = with_coordinates[{'neighbourhood','municipality', 'a_00_14', 'births', 'rental_perc','play_demand', 'latitude', 'longitude','play_distance'}]
play_output_list = play_output_list[play_output_list['play_demand'] > play_demand_threshold] 
play_output_list = play_output_list.sort_values(by=['play_demand'], ascending=False)
#play_output_list

C:\Users\camil\AppData\Local\Temp\ipykernel_14452\686637456.py:3: FutureWarning:

Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.



In [81]:
park_demand_threshold = 0.7
park_output_list = with_coordinates[{'neighbourhood','municipality', 'households_with_children', 'high_edu', 'a_00_14', 'a_45_64', 'females', 'not_married', 'park_demand', 'latitude', 'longitude', 'park_distance'}]
park_output_list = park_output_list[park_output_list['park_demand'] > park_demand_threshold] 
park_output_list = park_output_list.sort_values(by=['park_demand'], ascending=False)
#park_output_list

C:\Users\camil\AppData\Local\Temp\ipykernel_14452\1619924123.py:2: FutureWarning:

Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.



In [82]:
sport_demand_threshold = 0.6
sport_output_list = with_coordinates[{'neighbourhood','municipality', 'a_00_14', 'a_15_24', 'a_25_44', 'a_45_64', 'sport_demand', 'latitude', 'longitude', 'sport_distance'}]
sport_output_list = sport_output_list[sport_output_list['sport_demand'] > sport_demand_threshold] 
sport_output_list = sport_output_list.sort_values(by=['sport_demand'], ascending=False)
#sport_output_list

C:\Users\camil\AppData\Local\Temp\ipykernel_14452\4186675556.py:2: FutureWarning:

Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.



In [83]:
play_output_list.to_csv('play_output_list.csv', index=False)
park_output_list.to_csv('park_output_list.csv', index=False)
sport_output_list.to_csv('sport_output_list.csv', index=False)

## Dash

In [172]:
attributes = ['citizens', 'females', 'a_00_14', 'a_15_24', 'a_25_44', 'a_45_64', 'births', 'households_with_children',
'not_married', 'rental_perc', 'percentage_working', 'low_edu', 'mid_edu', 'high_edu', 'cars_per_household',]

Weight = ['citizens_weight', 'females_weight', 'a_00_14_weight', 'a_45_64_weight', 'births_weight', 'households_with_children_weight', 'single_weight', 'rental_weight',
               'working_weight', 'low_edu_weight', 'high_edu_weight', 'cars_weight'] 
    
Quantile = ['citizens_quantile', 'females_quantile', 'a_00_14_quantile', 'a_45_64_quantile', 'births_quantile', 'households_with_children_quantile', 'singles_quantile',
'rental_quantile', 'working_quantile',  'low_edu_quantile', 'high_edu_quantile', 'cars_quantile']
         

all_options = {
    'Weight': Weight,
    'Quantile': Quantile
}

# radio_output_quantile
radio_output_quantile = []
for i in all_options['Quantile']:
    radio_output_quantile.append(
    html.Label(i.replace('_',' ').capitalize()))
    radio_output_quantile.append(
    dcc.Slider(
    id=i,
    min=0,
    max=1,
    value = 0.5))
    
# radio_output_weight
radio_output_weight = []
for i in all_options['Weight']:
    radio_output_weight.append(
    html.Label(i.replace('_',' ').capitalize()))
    radio_output_weight.append(dcc.Input(id=i, value=1, type='number'))

In [176]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc
import plotly
import plotly.express as px
import plotly.graph_objects as go

# Data
#neighbourhoods = with_coordinates['neighbourhood'].unique()
gemeenten = with_coordinates['gemeente'].unique()
gemeenten = np.append(gemeenten,'Netherlands')

    
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)



app.layout = html.Div([
    html.H1('Blind Spot Map'),
    html.Div([
        # Map
        html.Div([
            html.B('Neighbourhood', style={'font-size': '18px'}),
            dcc.Dropdown(gemeenten, value='Netherlands', id='dropdown_province'),
            dcc.RadioItems(id='radio', options = ['Playgrounds', 'Sports Facilities','Parks & Nature'], value='Playgrounds',
                           inline=True, inputStyle={"margin-left": "20px"}),
            dcc.Graph(id='map'),
        ], style={'width': '65%', 'float': 'left', 'display': 'inline-block'}),
    
        # Manual inputs    
        html.Div([
            html.B('Demand Threshold', style={'font-size': '18px'}),
            dcc.Slider(min=0, max=1, value=0.7, id='slider_threshold'),
            html.Div([
                 html.H2('Quantile'),
                 html.Label('Citizens quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='citizens_quantile'),
                 html.Label('Females quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='females_quantile'),
                 html.Label('A 00 14 quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='a_00_14_quantile'),
                 html.Label('A 45 64 quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='a_45_64_quantile'),
                 html.Label('Births quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='births_quantile'),
                 html.Label('Households with children quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='households_with_children_quantile'),
                 html.Label('Singles quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='singles_quantile'),
                 html.Label('Rental quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='rental_quantile'),
                 html.Label('Working quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='working_quantile'),
                 html.Label('Low edu quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='low_edu_quantile'),
                 html.Label('High edu quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='high_edu_quantile'),
                 html.Label('Cars quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='cars_quantile'),
            ], className='row', style={'width': '50%', 'float': 'left', 'display': 'inline-block'}),
            
            html.Div([
                 html.H2('Weight'),
                 html.Label('Citizens weight'),
                 dcc.Input(value=1, type='number', id='citizens_weight'),
                 html.Label('Females weight'),
                 dcc.Input(value=1, type='number', id='females_weight'),
                 html.Label('A 00 14 weight'),
                 dcc.Input(value=1, type='number', id='a_00_14_weight'),
                 html.Label('A 45 64 weight'),
                 dcc.Input(value=1, type='number', id='a_45_64_weight'),
                 html.Label('Births weight'),
                 dcc.Input(value=1, type='number', id='births_weight'),
                 html.Label('Households with children weight'),
                 dcc.Input(value=1, type='number', id='households_with_children_weight'),
                 html.Label('Single weight'),
                 dcc.Input(value=1, type='number', id='single_weight'),
                 html.Label('Rental weight'),
                 dcc.Input(value=1, type='number', id='rental_weight'),
                 html.Label('Working weight'),
                 dcc.Input(value=1, type='number', id='working_weight'),
                 html.Label('Low edu weight'),
                 dcc.Input(value=1, type='number', id='low_edu_weight'),
                 html.Label('High edu weight'),
                 dcc.Input(value=1, type='number', id='high_edu_weight'),
                 html.Label('Cars weight'),
                 dcc.Input(value=1, type='number', id='cars_weight'),
            ], className='row', style={'width': '50%', 'float': 'right', 'display': 'inline-block'}),
            
            html.Br(),
            html.Hr(),
            
            html.Button('Submit', id='submit-val', n_clicks=0),
        ], className='row', style={'width': '25%', 'float': 'right', 'display': 'inline-block'}),
        
    ], className='row'),
    
    # Down Table
    html.Div([
        dbc.Container(id='tbl_out')
    ], className='row'),
          
])
@app.callback(
    Output("map", "figure"),
    Output('tbl_out', 'children'),
    Input("radio", "value"), 
    Input('dropdown_province', 'value'),
    Input('map', 'clickData'),
    Input('slider_threshold', 'value'),   
    # 12 weights
    Input('citizens_weight', 'value'),
    Input('females_weight', 'value'),
    Input('a_00_14_weight', 'value'),
    Input('a_45_64_weight', 'value'),
    Input('births_weight', 'value'),
    Input('households_with_children_weight', 'value'),
    Input('single_weight', 'value'),
    Input('rental_weight', 'value'),
    Input('working_weight', 'value'),
    Input('low_edu_weight', 'value'),
    Input('high_edu_weight', 'value'),
    Input('cars_weight', 'value'),
    # 12 Quantiles
    Input('citizens_quantile', 'value'),
    Input('females_quantile', 'value'),
    Input('a_00_14_quantile', 'value'),
    Input('a_45_64_quantile', 'value'),
    Input('births_quantile', 'value'),
    Input('households_with_children_quantile', 'value'),
    Input('singles_quantile', 'value'),
    Input('rental_quantile', 'value'),
    Input('working_quantile', 'value'),
    Input('low_edu_quantile', 'value'),
    Input('high_edu_quantile', 'value'),
    Input('cars_quantile', 'value'),  
)
def update_map(radio_input, province, clickData, slider_threshold,
               citizens_weight, females_weight, a_00_14_weight, a_45_64_weight, births_weight, households_with_children_weight,
               single_weight, rental_weight, working_weight, low_edu_weight, high_edu_weight, cars_weight,
               
               citizens_quantile, females_quantile, a_00_14_quantile, a_45_64_quantile, births_quantile, households_with_children_quantile,
               singles_quantile, rental_quantile, working_quantile, low_edu_quantile, high_edu_quantile, cars_quantile):
    
    df = with_coordinates
    hover_name = 'neighbourhood'
    zoom = 7
    if province != 'Netherlands':
        df = df[df['gemeente']==province]
        zoom = 11
    if(radio_input == "Playgrounds"):
        color = 'play_demand'
        size = 'play_demand'
        hover_data = ["play_demand", "play_distance"]

    elif (radio_input == "Parks & Nature"):
        color = 'park_demand'
        size = 'park_demand'
        hover_data = ["park_demand", "park_distance"]

    else:
        color = 'sport_demand'
        size = 'sport_demand'
        hover_data = ["sport_demand", "sport_distance"]

    fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color=color, size=size, mapbox_style = "open-street-map",
                          height = 900, zoom = zoom, opacity = 0.9, hover_name = hover_name, hover_data = hover_data, color_continuous_scale = plotly.colors.sequential.Purp)
    fig.update_layout(clickmode='event+select')
    
    # Table
    if clickData == None:
        table_out = []
    else:
        neighbourhood = dict(clickData['points'][0])['hovertext']
        df_table = df[df['neighbourhood'] == neighbourhood]
        df_table = df_table[['municipality', 'citizens', 'females', 'a_00_14', 'a_15_24', 'a_25_44', 'a_45_64', 
        'births', 'households_with_children', 'not_married', 'rental_perc', 'percentage_working', 'low_edu', 'mid_edu', 'high_edu', 'cars_per_household',
        'play_demand', 'sport_demand', 'park_demand', 'sport_distance', 'play_distance', 'park_distance']]
        table_out = [dbc.Label(neighbourhood), dash_table.DataTable(df_table.to_dict('records'),[{"name": i, "id": i} for i in df_table.columns], id='tbl')]
    
    return fig, table_out

@app.callback(
    Output('from_radio_chained', 'children'),
    Input('radio_chained', 'value'),
    Input('submit-val', 'n_clicks'),
)
def set_cities_options(selected_radio, n_clicks):
    radio_output = []
    if selected_radio == 'Quantile':
        radio_output = radio_output_quantile
    else:
        radio_output = radio_output_weight
    return radio_output

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


D:\Programs\anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [148]:
radio_output_quantile

[Label('Citizens quantile'),
 Slider(min=0, max=1, value=0.5, id='citizens_quantile'),
 Label('Females quantile'),
 Slider(min=0, max=1, value=0.5, id='females_quantile'),
 Label('A 00 14 quantile'),
 Slider(min=0, max=1, value=0.5, id='a_00_14_quantile'),
 Label('A 45 64 quantile'),
 Slider(min=0, max=1, value=0.5, id='a_45_64_quantile'),
 Label('Births quantile'),
 Slider(min=0, max=1, value=0.5, id='births_quantile'),
 Label('Households with children quantile'),
 Slider(min=0, max=1, value=0.5, id='households_with_children_quantile'),
 Label('Singles quantile'),
 Slider(min=0, max=1, value=0.5, id='singles_quantile'),
 Label('Rental quantile'),
 Slider(min=0, max=1, value=0.5, id='rental_quantile'),
 Label('Working quantile'),
 Slider(min=0, max=1, value=0.5, id='working_quantile'),
 Label('Low edu quantile'),
 Slider(min=0, max=1, value=0.5, id='low_edu_quantile'),
 Label('High edu quantile'),
 Slider(min=0, max=1, value=0.5, id='high_edu_quantile'),
 Label('Cars quantile'),
 Slide

In [149]:
radio_output_weight

[Label('Citizens weight'),
 Input(value=1, type='number', id='citizens_weight'),
 Label('Females weight'),
 Input(value=1, type='number', id='females_weight'),
 Label('A 00 14 weight'),
 Input(value=1, type='number', id='a_00_14_weight'),
 Label('A 45 64 weight'),
 Input(value=1, type='number', id='a_45_64_weight'),
 Label('Births weight'),
 Input(value=1, type='number', id='births_weight'),
 Label('Households with children weight'),
 Input(value=1, type='number', id='households_with_children_weight'),
 Label('Single weight'),
 Input(value=1, type='number', id='single_weight'),
 Label('Rental weight'),
 Input(value=1, type='number', id='rental_weight'),
 Label('Working weight'),
 Input(value=1, type='number', id='working_weight'),
 Label('Low edu weight'),
 Input(value=1, type='number', id='low_edu_weight'),
 Label('High edu weight'),
 Input(value=1, type='number', id='high_edu_weight'),
 Label('Cars weight'),
 Input(value=1, type='number', id='cars_weight')]

In [111]:
for metric in sorted(val_metrics.keys()):
            m = val_metrics[metric]
            data.append(np.mean(m))
            index.append(metric + "_mean")
            data.append(np.std(m))
            index.append(metric + "_std")

Index(['neighbourhood', 'municipality', 'citizens', 'females', 'a_00_14',
       'a_15_24', 'a_25_44', 'a_45_64', 'births', 'households_with_children',
       'not_married', 'rental_perc', 'percentage_working', 'low_edu',
       'mid_edu', 'high_edu', 'avg_income', 'cars_per_household',
       'postal_code_center', 'demand_a_00_14', 'demand_a_45_64',
       'demand_households_with_children', 'demand_cars', 'demand_births',
       'demand_rental', 'demand_females_perc', 'demand_singles_perc',
       'demand_low_edu', 'demand_high_edu', 'demand_percentage_working',
       'demand_citizens', 'play_demand', 'sport_demand', 'park_demand',
       'sport_distance', 'play_distance', 'park_distance', 'id', 'postcode',
       'woonplaats', 'alternatieve_schrijfwijzen', 'gemeente', 'provincie',
       'netnummer', 'latitude', 'longitude', 'soort'],
      dtype='object')

In [137]:
app.layout = html.Div([

    html.Button('Submit', id='submit-val', n_clicks=0),
])


@app.callback(
    
    Input('submit-val', 'n_clicks'),
)
def update_output(n_clicks, value):
    return 'The input value was "{}" and the button has been clicked {} times'.format(
        value,
        n_clicks
    )

Dash app running on http://127.0.0.1:8050/


pp.layout = html.Div(children=[
    html.Label("Treshold"),
    dcc.Slider(
        min=0,
        max=10,
        marks={i: str(i) for i in range(1, 10)},
        value=5
    ),


    html.Label("Attribute"),
    dcc.Dropdown(attributes),

    html.Label("Manual Inputs"),
    html.Div(children=[
        html.Label("Percentile"),
        dcc.Slider(
            id='percentile',
            min=0,
            max=10,
            marks={i: str(i) for i in range(1, 10)},
            value=default_percentile
        ),

        html.Label("Weight"),
        dcc.Input(id='weight', value=default_weight, type='number'),

        html.Br(),
        html.Button(id='reset', children='Reset', className="pure-button pure-button-primary")
    ], style={'border': '1px solid black', 'padding': 10})
], className='pure-form')

@app.callback(
    Output(component_id='weight', component_property='value'),
    Input(component_id='reset', component_property='n_clicks')
)
def reset_weight(n_clicks):
    return default_weight


@app.callback(
    Output(component_id='percentile', component_property='value'),
    Input(component_id='reset', component_property='n_clicks')
)
def reset_percentile(n_clicks):
    return default_percentile